# Phase 1: Environment Setup and Model Loading

## Overview
This notebook sets up the Google Colab environment for the CCoP 2.0 fine-tuning project.

### Objectives:
1. ✅ Clone project repository from GitHub
2. ✅ Install required dependencies
3. ✅ Set up GPU environment
4. ✅ Load and test Llama-Primus-Reasoning model
5. ✅ Configure API access for comparative models

### Requirements:
- GPU Runtime (T4 or A100 recommended)
- Sufficient disk space (~50GB)
- Internet access for model downloads

---

## 1. GPU Environment Check

In [ ]:
# Check GPU availability and specifications
import torch
import psutil
import os

print("🔧 GPU Environment Check")
print("=" * 40)

# GPU Check
if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"✅ CUDA Version: {torch.version.cuda}")
else:
    print("❌ No GPU available. Please enable GPU runtime.")
    print("Go to Runtime > Change runtime type > GPU")

# System Memory
memory_gb = psutil.virtual_memory().total / 1024**3
print(f"✅ System Memory: {memory_gb:.1f} GB")

# Disk Space
disk_usage = psutil.disk_usage('/')
free_gb = disk_usage.free / 1024**3
print(f"✅ Available Disk Space: {free_gb:.1f} GB")

if free_gb < 50:
    print("⚠️  Warning: Less than 50GB available. Consider clearing Colab files.")

## 2. Clone Project Repository

In [ ]:
# Clone the project repository
import os

# Repository configuration
REPO_URL = "https://github.com/sagerstack/primus-ccop-fine-tuning.git"
BRANCH = "feature/phase1-setup"  # Change to "main" for production
PROJECT_DIR = "/content/studio-ssdlc"

print("📥 Cloning Project Repository")
print("=" * 40)

# Remove existing directory if it exists
if os.path.exists(PROJECT_DIR):
    !rm -rf {PROJECT_DIR}

# Clone the repository
print(f"Cloning {REPO_URL} (branch: {BRANCH})...")
!git clone -b {BRANCH} {REPO_URL} {PROJECT_DIR}

# Change to project directory
os.chdir(PROJECT_DIR)
print(f"✅ Repository cloned to: {PROJECT_DIR}")
print(f"✅ Working directory: {os.getcwd()}")

# Verify repository structure
print("\n📁 Repository Structure:")
!ls -la

# Show key directories
print("\n📂 Key Directories:")
for dir_name in ['src', 'data', 'config', 'colab', 'benchmarks']:
    if os.path.exists(dir_name):
        print(f"✅ {dir_name}/")
    else:
        print(f"❌ {dir_name}/ (missing)")

## 3. Install Dependencies

In [ ]:
# Install required dependencies
print("📦 Installing Dependencies")
print("=" * 40)

# Core ML libraries
print("Installing PyTorch and CUDA libraries...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Transformers and ML libraries
print("Installing Transformers and ML libraries...")
!pip install transformers accelerate bitsandbytes datasets tokenizers

# Fine-tuning libraries
print("Installing fine-tuning libraries (QLoRA/PEFT)...")
!pip install peft trl

# Google Cloud and Drive APIs
print("Installing Google Cloud APIs...")
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install google-cloud-storage PyDrive2

# API integrations
print("Installing API client libraries...")
!pip install openai anthropic requests httpx

# Data processing and utilities
print("Installing data processing libraries...")
!pip install pandas numpy scikit-learn matplotlib seaborn tqdm rich

# Configuration and environment
print("Installing configuration libraries...")
!pip install pydantic pydantic-settings pyyaml python-dotenv

print("✅ All dependencies installed successfully!")

## 4. Environment Configuration

In [ ]:
# Set up environment configuration
import os
import yaml
from pathlib import Path

print("⚙️  Environment Configuration")
print("=" * 40)

# Load configuration
config_path = Path("config/environment/development.yaml")
if config_path.exists():
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    print("✅ Configuration loaded successfully")
    print(f"✅ Environment: {config['app']['environment']}")
    print(f"✅ App Name: {config['app']['name']}")
else:
    print("❌ Configuration file not found")
    config = {}

# Create necessary directories
dirs_to_create = [
    "data/benchmark",
    "models/base",
    "models/checkpoints",
    "benchmarks/results",
    "logs"
]

for dir_path in dirs_to_create:
    Path(dir_path).mkdir(parents=True, exist_ok=True)
    print(f"✅ Created directory: {dir_path}")

# Set up Python path
import sys
if "/content/studio-ssdlc/src" not in sys.path:
    sys.path.insert(0, "/content/studio-ssdlc/src")
    print("✅ Added src/ to Python path")

## 5. Model Setup and Authentication

In [ ]:
# Set up model authentication
from huggingface_hub import notebook_login
import os

print("🔐 Model Authentication Setup")
print("=" * 40)

# Hugging Face authentication
print("\n🤗 Hugging Face Authentication:")
print("Please enter your Hugging Face token when prompted.")
print("You can get a token from: https://huggingface.co/settings/tokens")

# Note: Uncomment the following line and run it to login
# notebook_login()

# For now, we'll check if token is already available
if os.environ.get('HUGGINGFACE_HUB_TOKEN'):
    print("✅ Hugging Face token found in environment")
else:
    print("⚠️  Please set HUGGINGFACE_HUB_TOKEN or run notebook_login()")

# Check API keys (these should be set as environment variables)
print("\n🔑 API Keys Check:")
api_keys = {
    'OpenAI': os.environ.get('OPENAI_API_KEY'),
    'Anthropic': os.environ.get('ANTHROPIC_API_KEY'),
    'DeepSeek': os.environ.get('DEEPSEEK_API_KEY')
}

for api, key in api_keys.items():
    if key:
        print(f"✅ {api} API key: {'*' * 10}{key[-4:] if len(key) > 4 else key}")
    else:
        print(f"⚠️  {api} API key: Not set")

print("\n💡 To set API keys in Colab:")
print("1. Click on the key icon in the left sidebar")
print("2. Add new secret for each API key")
print("3. Enable notebook access to the secrets")

## 6. Load Llama-Primus-Reasoning Model

In [ ]:
# Load Llama-Primus-Reasoning model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gc
from tqdm.auto import tqdm

print("🤖 Loading Llama-Primus-Reasoning Model")
print("=" * 40)

# Model configuration
MODEL_NAME = "trendmicro-ailab/Llama-Primus-Reasoning"
DEVICE = "auto"
TORCH_DTYPE = torch.float16

# Quantization configuration (for memory efficiency)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

print(f"📋 Model Configuration:")
print(f"  - Model: {MODEL_NAME}")
print(f"  - Device: {DEVICE}")
print(f"  - Data Type: {TORCH_DTYPE}")
print(f"  - 4-bit Quantization: Enabled")

try:
    # Load tokenizer
    print("\n📥 Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
        use_fast=True
    )
    
    # Set padding token if not present
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print("✅ Tokenizer loaded successfully")
    print(f"   - Vocabulary size: {len(tokenizer)}")
    print(f"   - Pad token: {tokenizer.pad_token}")

    # Clear cache before loading model
    gc.collect()
    torch.cuda.empty_cache()

    # Load model
    print("\n🧠 Loading model (this may take 5-10 minutes)...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map=DEVICE,
        torch_dtype=TORCH_DTYPE,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

    print("✅ Model loaded successfully!")
    print(f"   - Model parameters: {model.num_parameters():,}")
    print(f"   - Device mapping: {model.hf_device_map}")
    
    # Memory usage
    if torch.cuda.is_available():
        memory_used = torch.cuda.memory_allocated() / 1024**3
        memory_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"   - GPU Memory: {memory_used:.1f}/{memory_total:.1f} GB ({memory_used/memory_total*100:.1f}%)")

except Exception as e:
    print(f"❌ Error loading model: {str(e)}")
    print("\n💡 Possible solutions:")
    print("1. Check your Hugging Face authentication")
    print("2. Ensure you have sufficient GPU memory")
    print("3. Try restarting the runtime")
    model, tokenizer = None, None

## 7. Test Model Functionality

In [ ]:
# Test model functionality
if model is not None and tokenizer is not None:
    print("🧪 Testing Model Functionality")
    print("=" * 40)

    # Test prompts
    test_prompts = [
        "What is cybersecurity?",
        "Explain the concept of risk assessment in simple terms.",
        "What are the basic principles of network security?"
    ]

    for i, prompt in enumerate(test_prompts, 1):
        print(f"\n📝 Test {i}: {prompt}")
        
        try:
            # Tokenize input
            inputs = tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=512,
                padding=True
            ).to(model.device)

            # Generate response
            with torch.no_grad():
                outputs = model.generate(
                    inputs.input_ids,
                    max_new_tokens=150,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            # Decode response
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Extract only the generated part
            if prompt in response:
                generated_text = response[len(prompt):].strip()
            else:
                generated_text = response.strip()

            print(f"💬 Response: {generated_text[:200]}{'...' if len(generated_text) > 200 else ''}")
            
            # Clean up memory
            del inputs, outputs
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"❌ Error: {str(e)}")

    print("\n✅ Model functionality test completed!")
else:
    print("❌ Model not loaded. Please complete the model loading step first.")

## 8. Next Steps

### ✅ Completed in this notebook:
1. ✅ GPU environment verification
2. ✅ Repository cloned from GitHub
3. ✅ Dependencies installed
4. ✅ Environment configured
5. ✅ Llama-Primus-Reasoning model loaded
6. ✅ Model functionality tested

### 📋 Next notebooks to run:
1. **`02_benchmark_testing.ipynb`** - Run Phase 2-3 benchmarks
2. **`03_results_analysis.ipynb`** - Analyze benchmark results

### 🔧 Setup Required:
- Set up API keys in Colab secrets (OpenAI, Anthropic, DeepSeek)
- Configure Google Drive access for result storage
- Ensure Hugging Face authentication is working

### 📊 Ready for Phase 2:
The environment is now ready for Phase 2 baseline screening with 40 test cases (B1-B6 benchmarks).